In [228]:
# Importing necessary libraries
import jax

import jax.numpy as jnp
import numpy as np
import cvxpy as cvx
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots
from stl import mesh
import plotly.graph_objects as go
import tqdm

In [229]:
# Constants
mu_earth    = 3.986004418e14 # m^3/s^2
J2_earth    = 0.00108263
r_earth     = 6378.138e3 # m

Code to convert keplerian elements $\xleftrightarrow{\text{}}$ cartesian vectors taken from my personal project code developed from previous course from undergrad.
University of Illinois AE402 - Orbital Mechanics

In [230]:
def unit(v):
    """Returns the unit vector of v."""
    return v / np.linalg.norm(v)
def skew(v):
    return np.array([[0,    -v[2],  v[1]],
                     [v[2], 0,      -v[0]],
                     [-v[1], v[0],   0]])

In [231]:
def keplerian_to_state_vec(kep, mu):
    '''
    Converts keplerian elements to cartesian state vector
    :param kep: list of keplerian elements [a, e, i, raan, omega, f]
    :param mu: gravitational parameter of the central body
    :return: state vector [x, y, z, vx, vy, vz]
    '''
    pos = position(kep, mu)
    vel = velocity(kep, mu)

    state_vec = np.array([pos[0], pos[1], pos[2], vel[0], vel[1], vel[2]])
    return state_vec

def angular_momentum_from_OE(a, e, mu):
    '''
    Calculates the angular momentum of an orbit given the semi-major axis and eccentricity
    :param a: semi-major axis
    :param e: eccentricity
    :param mu: gravitational parameter of the central body
    :return: angular momentum
    '''
    if e >=0 and e < 1:
        return np.sqrt(mu*a*(1-e**2))
    elif e < 0:
        return np.sqrt(mu*a*(e**2-1))
    
def position(kep, mu):
    '''
    Converts keplerian elements to cartesian position vector
    :param kep: list of keplerian elements [a, e, i, raan, omega, f]
    :param mu: gravitational parameter of the central body
    :return: position vector [x, y, z]
    '''
    a = kep[0]
    e = kep[1]
    i = np.radians(kep[2])
    raan = np.radians(kep[3])
    omega = np.radians(kep[4])
    f = np.radians(kep[5])

    r = a*(1-e**2)/(1+e*np.cos(f))
    theta = omega + f
    pos = r * np.array([np.cos(theta)*np.cos(raan) - np.cos(i)*np.sin(raan)*np.sin(theta),
                        np.cos(theta)*np.sin(raan) + np.cos(i)*np.cos(raan)*np.sin(theta),
                        np.sin(i)*np.sin(theta)])
    return pos

def velocity(kep, mu):
    '''
    Converts keplerian elements to cartesian velocity vector
    :param kep: list of keplerian elements [a, e, i, raan, omega, f]
    :param mu: gravitational parameter of the central body
    :return: velocity vector [vx, vy, vz]
    '''
    a = kep[0]
    e = kep[1]
    inc = np.radians(kep[2])
    raan = np.radians(kep[3])
    omega = np.radians(kep[4])
    f = np.radians(kep[5])

    theta = omega + f

    h = angular_momentum_from_OE(a, e, mu)
    theta = omega + f
    vel_vec = mu/h*np.array([-(np.cos(raan)*(np.sin(theta) + e*np.sin(omega)) + np.sin(raan)*(np.cos(theta)+ e*np.cos(omega))*np.cos(inc)),
                             -(np.sin(raan)*(np.sin(theta) + e*np.sin(omega)) - np.cos(raan)*(np.cos(theta) + e*np.cos(omega))*np.cos(inc)),
                             (np.cos(theta) + e*np.cos(omega))*np.sin(inc)])
                         
    return vel_vec

def semimajor_axis(state, mu):
    '''
    Calculates the semimajor axis of the orbit given a state
    vector and gravitational parameter
    :param state: state vector of the form [x, y, z, vx, vy, vz]
    :param mu: gravitaitonal parameter of the central body
    :return: semimajor axis (a)
    '''
    r = np.linalg.norm(state[0:3])
    v = np.linalg.norm(state[3:6])
    if r == 0:
        return 0
    a = 1/(2/r - v**2/mu)
    return a

def gravity(r, mu):
    '''
    Calculates the gravitational acceleration at a point
    :param r: position vector
    :param mu: gravitational parameter of the central body
    :return: gravitational acceleration vector
    '''
    return -mu*r/(np.linalg.norm(r)**3)

def J2_perturbation(r, mu, R, J2):
    '''
    Calculates the J2 perturbation acceleration at a point
    :param r: position vector
    :param mu: gravitational parameter of the central body
    :param R: radius of the central body
    :param J2: J2 coefficient of the central body
    :return: J2 perturbation acceleration vector
    '''
    r_mag = np.linalg.norm(r)
    J2_pert = -3/2 * J2 * mu / (r_mag**2) * (R/r_mag)**2 * np.array([(1 - 5*(r[2]/r_mag)**2)*r[0]/r_mag, 
                                                                           (1 - 5*(r[2]/r_mag)**2)*r[1]/r_mag, 
                                                                           (3 - 5*(r[2]/r_mag)**2)*r[2]/r_mag])
    return J2_pert


In [232]:
def state_dot(t, state, mu, J2, R, accel_cmds):
    '''
    Calculates the derivative of the state vector
    :param t: current time
    :param state: current state vector [x, y, z, vx, vy, vz, m]
    :param mu: gravitational parameter of the central body
    :return: derivative of the state vector [x_dot, y_dot, z_dot, vx_dot, vy_dot, vz_dot]
    '''
    leader_state = state[0:6]
    follower_state = state[6:12]

    leader_pos = leader_state[0:3]
    leader_vel = leader_state[3:6]

    follower_pos = follower_state[0:3]
    follower_vel = follower_state[3:6]

    grav1 = gravity(leader_pos, mu)
    p_1 = J2_perturbation(leader_pos, mu, R, J2)
    v1_dot = grav1 + p_1 + accel_cmds[0]

    grav2 = gravity(follower_pos, mu)
    p_2 = J2_perturbation(leader_pos, mu, R, J2)
    v2_dot = grav2 + p_2 + accel_cmds[1]

    state_dot = np.concatenate((leader_vel, v1_dot, follower_vel, v2_dot))

    return state_dot

In [233]:
def get_hill_to_inertial(state):
    '''
    Calculates the rotation matrix from the Hill frame to the inertial frame
    :param state: current state vector [x, y, z, vx, vy, vz]
    :return: rotation matrix from the Hill frame to the inertial frame
    '''
    r = state[0:3]
    v = state[3:6]
    h = np.cross(r, v)

    r_normalized    = r/np.linalg.norm(r)
    h_normalized    = h/np.linalg.norm(h)

    # Rotation matrix from Hill to Inertial
    R_hill_to_inertial = np.array([r_normalized, np.cross(h_normalized, r_normalized), h_normalized]).T
    
    a = semimajor_axis(state, mu_earth)
    n =  np.sqrt(mu_earth/a**3)
    omega_hill = np.array([0, 0, n])
    
    return R_hill_to_inertial, omega_hill

In [234]:
def inertial_to_hill(inertial_state):
    '''
    Converts the inertial state to the Hill frame
    :param inertial_state: current state vector of the leader concatenated with the current state vector of the chaser 
                            in the inertial frame [x, y, z, vx, vy, vz]
    :return: current state vector of the chaser in the Hill frame [x, y, z, vx, vy, vz]
    '''
    state_lead = inertial_state[0:6]
    pos_lead = state_lead[0:3]
    vel_lead = state_lead[3:6]

    state_chaser = inertial_state[6:12]
    pos_chaser = state_chaser[0:3]
    vel_chaser = state_chaser[3:6]

    rel_pos = pos_chaser - pos_lead
    rel_vel = vel_chaser - vel_lead

    R_hill_to_inertial, omega_hill = get_hill_to_inertial(state_lead)

    hill_pos = R_hill_to_inertial.T@(rel_pos)
    hill_vel = R_hill_to_inertial.T@(rel_vel) - np.cross(omega_hill, R_hill_to_inertial.T@(rel_pos))
    return np.hstack((hill_pos, hill_vel))
    
def hill_to_inertial(hill_state, state_lead):
    '''
    Converts the Hill state to the inertial frame
    :param hill_state: current state vector of the chaser in the Hill frame [x, y, z, vx, vy, vz]
    :param state_lead: current state vector of the leader in the inertial frame [x, y, z, vx, vy, vz]
    :return: current state vector of the chaser in the inertial frame [x, y, z, vx, vy, vz]
    '''
    hill_pos = hill_state[0:3]
    hill_vel = hill_state[3:6]

    R_hill_to_inertial, omega_hill = get_hill_to_inertial(state_lead)
    
    inertial_pos    = (R_hill_to_inertial @ hill_pos) + state_lead[0:3]
    inertial_vel    = (R_hill_to_inertial @ (hill_vel + np.cross(omega_hill, hill_pos))) + state_lead[3:6]

    return np.hstack((inertial_pos, inertial_vel))

In [235]:
def update_hists(state, idx, leader_array, chaser_array, hill_array):
    state_lead      = state[0:6]
    state_chaser    = state[6:12]
    hill_state      = inertial_to_hill(state)
    
    leader_array[idx,:] = state_lead
    chaser_array[idx,:] = state_chaser
    hill_array[idx,0:3] = hill_state[0:3]
    hill_array[idx,3:6] = hill_state[3:6]

In [236]:
@jax.jit
def obstacle_constraint(state, obstacle, radius):
    return jnp.linalg.norm(state[0:3] - obstacle[0:3]) - radius

# jit the linearize constraint functions to make it run faster
linearize_obstacle = jax.jit(jax.vmap(jax.grad(obstacle_constraint), in_axes=[0, None, None]))

In [237]:
def solve_keep_out(initial_state, planning_horizon, A, B, Q, R, des_state, dt, keep_out_rad):
    solver  = cvx.CLARABEL
    x       = cvx.Variable((planning_horizon+1, 6))
    u       = cvx.Variable((planning_horizon, R.shape[0]))
    slack   = cvx.Variable(1)

    Gs = [cvx.Parameter([6]) for _ in range(planning_horizon+1)] # parameters for linearized constraints
    hs = [cvx.Parameter(1) for _ in range(planning_horizon+1)] # parameters for linearized constraints

    x_previous = cvx.Parameter([planning_horizon+1, 6]) # parameter for previous solution
    u_previous = cvx.Parameter([planning_horizon, 3]) # parameter for previous solution

    slack_penalty   = 100000. # coefficient for slack variable

    objective           = slack_penalty * slack**2 + cvx.quad_form(x[-1, :] - des_state, Q)
    constraints         = [x[0, :] == initial_state]
    constraints_no_obs  = [x[0, :] == initial_state]
    for t in range(planning_horizon):
        objective   += cvx.quad_form(u[t, :], R)

        constraints += [x[t+1, :] == x[t, : ] + (A @ x[t, :] + B @ u[t, :])*dt]
        constraints += [cvx.norm(u[t, 0:3], 'inf') <= 0.032]
        constraints += [Gs[t] @ (x[t]) + hs[t] >= -slack] # linearized collision avoidance constraint
        
        constraints_no_obs += [x[t+1, :] == x[t, : ] + (A @ x[t, :] + B @ u[t, :])*dt]
        constraints_no_obs += [cvx.norm(u[t, 0:3], 'inf') <= 0.032]

    prob_no_obs = cvx.Problem(cvx.Minimize(objective), constraints=constraints_no_obs)
    prob_no_obs.solve(solver=solver) # solve the QP

    x_previous.value    = x.value
    u_previous.value    = u.value
    sqp_x_solutions       = [x_previous.value]
    sqp_u_solutions       = [u_previous.value]

    problem_obs = cvx.Problem(cvx.Minimize(objective), constraints=constraints)

    for i in range(2):

        Gs_value = linearize_obstacle(x_previous.value, jnp.zeros(6), keep_out_rad) # Linearizes obstacle constraint 
        hs_value = jax.vmap(obstacle_constraint, [0, None, None])(x_previous.value, jnp.zeros(6), keep_out_rad) - jax.vmap(jnp.dot, [0, 0])(Gs_value, x_previous.value) # Sets obstacle location relative to robot
        
        for i in range(planning_horizon): # Stores linearized dynamics matrices and vectors in arrays
            Gs[i].value = np.array(Gs_value[i])
            hs[i].value = np.array(hs_value[i:i+1])
        Gs[planning_horizon].value = np.array(Gs_value[planning_horizon]) # Sets the linearized obstacle constraint matrices at the last time step
        hs[planning_horizon].value = np.array(hs_value[planning_horizon:planning_horizon+1]) # Sets the linearized obstacle constraint vectors at the last time step

        problem_obs.solve(solver=solver) # solve the QP

        x_previous.value = np.array(x.value)
        u_previous.value = np.array(u.value)

        sqp_x_solutions.append(np.array(x_previous.value))
        sqp_u_solutions.append(np.array(u_previous.value))
    
    return sqp_x_solutions[-1], sqp_u_solutions[-1]

In [238]:
def solve_approach_input(initial_state, planning_horizon, A, B, Q, R, des_state, dt, appch_cone_ang, appch_cone_dir):
    solver  = cvx.CLARABEL
    x       = cvx.Variable((planning_horizon+1, initial_state.shape[0]))
    u       = cvx.Variable((planning_horizon,3))

    theta   = np.deg2rad(appch_cone_ang)    # cone half-angle in radians

    cost        = cvx.quad_form(x[-1, :] - des_state, Q)
    constraints = [x[0, :] == initial_state]
    constraints += [(x[-1, 0:3].T - des_state[0:3]) @ appch_cone_dir >= 0]
    constraints += [cvx.norm((x[-1, 3:6] - des_state[3:6]).T @ appch_cone_dir, 2) <= 1]
    constraints += [cvx.norm(u[0, 0:3], 'inf') <= 0.032]
    for t in range(planning_horizon):
        cost        += cvx.quad_form(x[t, :] - des_state, Q) + cvx.quad_form(u[t, :], R)
        constraints += [x[t+1, :] == x[t, : ] + (A @ x[t, :] + B @ u[t, :])*dt]

        r           = x[t, 0:3] - des_state[0:3]
        constraints += [cvx.norm(r) * np.cos(theta/2) <= r.T @ appch_cone_dir]
        constraints += [r.T @ appch_cone_dir >= 0]
        constraints += [cvx.norm((x[t, 3:6] - des_state[3:6]).T @ appch_cone_dir, 2) <= 1]
        constraints += [cvx.norm(u[t, 0:3], 'inf') <= 0.032]

    lqr = cvx.Problem(cvx.Minimize(cost), constraints=constraints)
    lqr.solve(solver=solver)

    return x.value, u.value

In [239]:
def sim(init_state, times, rndz_dat):
    '''
    Simulates the spacecraft trajectory in the ECI frame
    :param init_states: initial states of the spacecrafts
    :param times: A vector of times at which the state is to be evaluated
    :return: The times, state_hist where
            the times is a vector of times at which the state is evaluated and
            the state is a 6x1 vector of the form [x, y, z, vx, vy, vz] in the ECI Frame
    '''

    # Unpack initial states
    state       = init_state

    # Unpack rendezvous event times
    t_rndz_init         = rndz_dat[0]
    waypoints           = rndz_dat[1]
    keep_out_rad        = rndz_dat[2]
    appch_cone_angle    = rndz_dat[3]
    appch_cone_dir      = rndz_dat[4]

    # Determine the time step
    dt          = times[1] - times[0]

    # Initialize flags for rendezvous initiation and completion
    rndz_mode  = 0
    replan_flag = False

    # Initialize history arrays to store states
    state_lead_hist         = np.zeros((len(times), 6))
    state_chaser_hist       = np.zeros((len(times), 6))
    state_chaser_hill_hist  = np.zeros((len(times), 6))
    input_hist              = np.zeros((len(times), 3))
    # Store initial states
    update_hists(state, 0, state_lead_hist, state_chaser_hist,  state_chaser_hill_hist)
    
    accel_cmds = np.zeros((2,3))

    state_seq = np.zeros((len(times), 6))
    idx_end_plan = len(state_seq)

    idx_end_keep_out = 0
    idx_end_rndz = 0

    replanning_horizon = int(500/dt)

    # Loop over time and propagate state forward
    for t in tqdm.tqdm(range(len(times)-1)):
        # Get the current and next time
        t_now   = times[t]
        t_next  = t_now + dt

        # Propagate the state forward in time based on the current state and thrust vector
        sol     = solve_ivp(state_dot, t_span=[t_now, t_next], y0=state, args=(mu_earth, J2_earth, r_earth, accel_cmds), method='RK45', rtol=1e-10, atol=1e-10)
        
        # Update the state, thrust and control history arrays
        if rndz_mode <=2:
            state   = sol.y[:,-1]
        else:
            state[0:6] = sol.y[:,-1][0:6]
            state[6:12] = hill_to_inertial(chaser_hill_state, state[0:6])

        # Unpack the state into leader and chaser states
        lead_state      = state[0:6]
        chaser_state    = state[6:12]

        # Get the hill frame state of the chaser
        chaser_hill_state   = inertial_to_hill(state)

        accel_cmd_chaser_hill = np.zeros(3)

        n = np.sqrt(mu_earth/semimajor_axis(lead_state, mu_earth)**3)

        A = np.array([[0,0, 0, 1, 0, 0],
                      [0, 0, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0, 1],
                      [3*n**2, 0, 0, 0, 2*n, 0],
                      [0, 0, 0, -2*n, 0, 0],
                      [0, 0, -n**2, 0, 0, 0]])
        
        B = np.array([[0, 0, 0],
                      [0, 0, 0],
                      [0, 0, 0],
                      [1, 0, 0],
                      [0, 1, 0],
                      [0, 0, 1]])
        
        if t_now >= t_rndz_init and rndz_mode == 0:
            # Set the flag and the index of the start of the rendezvous maneuver
            rndz_mode = 1
            replan_flag = True
            idx_start_rndz = t
        
        if rndz_mode == 1:
            target_state = waypoints[0]

            if replan_flag:
                Q = np.diag([1, 1, 1, 1, 1, 1])
                R = np.diag([1, 1, 1])

                # Replan the trajectory
                state_seq, accel_seq = solve_keep_out(chaser_hill_state, replanning_horizon, A, B, Q, R, target_state, dt, keep_out_rad)
                replan_flag = False
                if accel_seq is None:
                    accel_seq = np.zeros((replanning_horizon, 3))
                idx_end_plan = t+len(accel_seq)
            
            # Use the previously planned trajectory
            if t < idx_end_plan:
                idx_plan = (t - idx_end_plan + replanning_horizon)
                accel_cmd_chaser_hill = accel_seq[idx_plan]
            elif t == idx_end_plan:
                accel_cmd_chaser_hill = np.zeros(3)
            elif np.linalg.norm(chaser_hill_state[3:6] - target_state[3:6]) > 1:
                replan_flag = True
            else:
                # If the planning horizon is exceeded, stop the maneuver
                rndz_mode = 2
                replan_flag = True
                idx_end_keep_out = t

        if rndz_mode == 2:
            target_state = waypoints[1]
            replanning_horizon = int(60/dt)
            if replan_flag:
                Q = np.diag([1, 1, 1, 1, 1, 1])
                R = np.diag([1, 1, 1])
                # Replan the trajectory
                state_seq, accel_seq = solve_approach_input(chaser_hill_state, replanning_horizon, A, B, Q, R, target_state, dt, appch_cone_angle, appch_cone_dir)
                replan_flag = False
                if accel_seq is None:
                    accel_seq = np.zeros((replanning_horizon, 3))
                idx_end_plan = t+len(accel_seq)

            # Use the previously planned trajectory
            if np.linalg.norm(chaser_hill_state[0:2] - target_state[0:2]) < 0.075:
                rndz_mode = 3
                replan_flag = False
                idx_end_rndz = t
                accel_cmd_chaser_hill = np.zeros(3)
            elif t < idx_end_plan:
                idx_plan = (t - idx_end_plan + replanning_horizon)
                accel_cmd_chaser_hill = accel_seq[idx_plan]
            elif t == idx_end_plan:
                accel_cmd_chaser_hill = np.zeros(3)
            elif np.linalg.norm(chaser_hill_state[0:2] - target_state[0:2]) > 0.075:
                replan_flag = True
            else:
                rndz_mode = 3
                replan_flag = False
                idx_end_rndz = t
                accel_cmd_chaser_hill = np.zeros(3)

        R_hill_to_inertial, omega_hill = get_hill_to_inertial(lead_state)
        accel_cmd_chaser = R_hill_to_inertial @ accel_cmd_chaser_hill
        input_hist[t, :] = accel_cmd_chaser_hill
        accel_cmds = np.vstack([np.zeros(3), accel_cmd_chaser])

        # Store the states in the history arrays
        update_hists(state, t+1, state_lead_hist, state_chaser_hist,  state_chaser_hill_hist)

    # If the simulation runs to completion, return the burn start index, time array, state, thrust and control history
    return times, state_lead_hist, state_chaser_hist, state_chaser_hill_hist, [idx_start_rndz, idx_end_keep_out, idx_end_rndz], input_hist

In [240]:
# Initial conditions
alt = 417*1000 # m
a_lead = r_earth + alt
kep = np.array([a_lead, 0.007, 51.6, 0, 0, 0])

# Convert keplerian elements to state vector
state_lead   = keplerian_to_state_vec(kep, mu_earth)

n = np.sqrt(mu_earth/a_lead**3)

# Initial conditions for chaser spacecraft
hill_pos        = np.array([-100, -500, 0])
hill_vel        = np.array([0, -2*n*hill_pos[0], 0])
hill_state      = np.hstack([hill_pos, hill_vel])

# Convert Hill frame to Inertial frame
state_chaser    = hill_to_inertial(hill_state, state_lead)

# Combine the states of the lead and chaser spacecraft
states          = np.hstack((state_lead, state_chaser))

# Times
t_rndz_init = 60
tf          = 890
dt          = 1
times       = np.arange(0, tf+dt, dt)

waypoints = np.array([[-3, 240, 0, 0, 0, 0],
                      [-3, 20, 0, 0, 0, 0]])
keep_out_rad = 200
approach_cone_angle = 10
approach_direction = np.array([0, 1, 0])  # Desired approach direction (unit vector)

rndz_dat  = [t_rndz_init, waypoints, keep_out_rad, approach_cone_angle, approach_direction]

# Simulate the trajectory
times, state_lead_hist, state_chaser_hist, state_chaser_hill_hist, rndz_idxs, inputs = sim(states, times, rndz_dat)

100%|██████████| 890/890 [00:13<00:00, 66.38it/s] 


In [241]:
def add_frame(fig, frame):
        fig.add_trace(go.Scatter3d(x=[0, frame[0,0]], 
                                 y=[0, frame[0,1]], 
                                 z=[0, frame[0,2]],
                                 mode="lines", line=dict(width=3, color="red"), showlegend=False))
        fig.add_trace(go.Scatter3d(x=[0, frame[1,0]],
                                 y=[0, frame[1,1]],
                                 z=[0, frame[1,2]],
                                 mode="lines", line=dict(width=3, color="green"), showlegend=False))
        fig.add_trace(go.Scatter3d(x=[0, frame[2,0]],
                                 y=[0, frame[2,1]],
                                 z=[0, frame[2,2]],
                                 mode="lines", line=dict(width=3, color="blue"), showlegend=False))
        fig.add_trace(go.Cone(x=[frame[0,0]],
                            y=[frame[0,1]],
                            z=[frame[0,2]],
                            u=[.3*frame[0,0]],
                            v=[.3*frame[0,1]],
                            w=[.3*frame[0,2]],
                            anchor = "tail",
                            hoverinfo =  None,
                            colorscale = [[0, "red"], [1, "red"]],
                            showscale = False))
        fig.add_trace(go.Cone(x=[frame[1,0]],
                            y=[frame[1,1]],
                            z=[frame[1,2]],
                            u=[.3*frame[1,0]],
                            v=[.3*frame[1,1]],
                            w=[.3*frame[1,2]],
                            anchor = "tail",
                            hoverinfo =  None,
                            colorscale = [[0, "green"], [1, "green"]],
                            showscale = False))
        fig.add_trace(go.Cone(x=[frame[2,0]],
                        y=[frame[2,1]],
                        z=[frame[2,2]],
                        u=[.3*frame[2,0]],
                        v=[.3*frame[2,1]],
                        w=[.3*frame[2,2]],
                        anchor = "tail",
                        hoverinfo =  None,
                        colorscale = [[0, "blue"], [1, "blue"]],
                        showscale = False),)
        
def make_sphere(x, y, z, radius, resolution=20):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + x
    Y = radius * np.sin(u)*np.sin(v) + y
    Z = radius * np.cos(v) + z
    points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()])
    return points.T

In [242]:
def stl2mesh3d(stl_mesh):
    # stl_mesh is read by nympy-stl from a stl file; it is  an array of faces/triangles (i.e. three 3d points) 
    # this function extracts the unique vertices and the lists I, J, K to define a Plotly mesh3d
    p, q, r = stl_mesh.vectors.shape #(p, 3, 3)
    # the array stl_mesh.vectors.reshape(p*q, r) can contain multiple copies of the same vertex;
    # extract unique vertices from all mesh triangles
    vertices, ixr = np.unique(stl_mesh.vectors.reshape(p*q, r), return_inverse=True, axis=0)
    I = np.take(ixr, [3*k for k in range(p)])
    J = np.take(ixr, [3*k+1 for k in range(p)])
    K = np.take(ixr, [3*k+2 for k in range(p)])
    IJK = np.vstack((I, J, K)).T
    return vertices, IJK

def model(model_file, axis_order=[0, 1, 2], scale=1.0):
    # Unpack model location and axis order
    model       = model_file

    # Get model data
    vehicle = mesh.Mesh.from_file(model)
    vehicle_points, IJK = stl2mesh3d(vehicle)

    # Swap the axes to match the vehicle
    vehicle_points = vehicle_points[:, axis_order]
    vehicle_points -= np.mean(vehicle_points, axis=0)

    # Centroid the vehicle
    min_x = np.min(vehicle_points[:,0])
    max_x = np.max(vehicle_points[:,0])
    min_y = np.min(vehicle_points[:,1])
    max_y = np.max(vehicle_points[:,1])
    min_z = np.min(vehicle_points[:,2])
    max_z = np.max(vehicle_points[:,2])

    range_x = max_x - min_x
    range_y = max_y - min_y
    range_z = max_z - min_z

    # Normalize the vehicle scale
    max_range       = max(range_x, range_y, range_z)
    vehicle_points  = vehicle_points/max_range*scale

    return [vehicle_points, IJK]

In [243]:
ISS_points, vertices = model("ISS.stl", axis_order=[1, 2, 0], scale=.1084)
Dragon_points, vertices_dragon = model("Dragon.stl", axis_order=[1, 2, 0], scale=.0081)

In [244]:
# Plot the entire simulation trajectory in the Hill frame (Leader)
fig     = go.Figure(data=[go.Scatter3d(x=state_chaser_hill_hist[:,0]/1000, 
                                       y=state_chaser_hill_hist[:,1]/1000, 
                                       z=state_chaser_hill_hist[:,2]/1000, 
                                       mode = 'lines', 
                                       name = 'Chaser Trajectory', 
                                       line = dict(width=5, 
                                                  color=times, 
                                                  colorscale= 'Matter'))])

fig.add_trace(go.Scatter3d(x=[state_chaser_hill_hist[0,0]/1000],
                            y=[state_chaser_hill_hist[0,1]/1000],
                            z=[state_chaser_hill_hist[0,2]/1000],
                            mode = 'markers',
                            name = 'Chaser Initial Position',
                            marker = dict(size=7)))

fig.add_trace(go.Scatter3d(x=[state_chaser_hill_hist[rndz_idxs[0],0]/1000],
                            y=[state_chaser_hill_hist[rndz_idxs[0],1]/1000],
                            z=[state_chaser_hill_hist[rndz_idxs[0],2]/1000],
                            mode = 'markers',
                            name = 'Chaser Position at Rendezvous Init',
                            marker = dict(size=5)))

fig.add_trace(go.Scatter3d(x=[state_chaser_hill_hist[rndz_idxs[1],0]/1000],
                            y=[state_chaser_hill_hist[rndz_idxs[1],1]/1000],
                            z=[state_chaser_hill_hist[rndz_idxs[1],2]/1000],
                            mode = 'markers',
                            name = 'Chaser Position at Docking Init',
                            marker = dict(size=5)))

fig.add_trace(go.Scatter3d(x=[state_chaser_hill_hist[rndz_idxs[2],0]/1000],
                            y=[state_chaser_hill_hist[rndz_idxs[2],1]/1000],
                            z=[state_chaser_hill_hist[rndz_idxs[2],2]/1000],
                            mode = 'markers',
                            name = 'Final Chaser Position',
                            marker = dict(size=7)))

fig.add_trace(go.Mesh3d(x=ISS_points[:,0],
                        y=ISS_points[:,1],
                        z=ISS_points[:,2],
                        i=vertices[:,0],
                        j=vertices[:,1],
                        k=vertices[:,2],
                        alphahull=10,
                        opacity=1.00,
                        color="silver", 
                        lighting=dict(diffuse=0.5), showlegend=False))

sphere = make_sphere(0, 0, 0, keep_out_rad/1000)
x, y, z = sphere[:,0], sphere[:,1], sphere[:,2]

fig.add_trace(go.Mesh3d(x=x, y=y, z=z, color="red", opacity=0.10, alphahull=0, name="Keep Out Sphere"))

theta = np.linspace(0, 2*np.pi, 100)
r = np.sin(np.radians(approach_cone_angle/2)) * keep_out_rad/1000
x = r * np.cos(theta) + waypoints[1,0]/1000
y = np.ones_like(x) * keep_out_rad/1000
z = r * np.sin(theta) + waypoints[1,2]/1000

x=np.append(x,waypoints[1,0]/1000)
y=np.append(y,waypoints[1,1]/1000)
z=np.append(z,waypoints[1,2]/1000)

fig.add_trace(go.Mesh3d(x=x, y=y, z=z, color="green", opacity=0.20, alphahull=0, name="Docking Corridor"))

add_frame(fig, np.eye(3)/4)

fig.update_layout(title="Rendezvous and Docking Trajectory in Hill Frame",
                  scene=dict(xaxis_title='Radial (km)', 
                             yaxis_title='Along-Track (km)', 
                             zaxis_title='Out-of-Plane (km)'),
                scene_camera=dict(up=dict(x=1, y=0, z=0),eye=dict(x=0, y=0, z=-2.3)))
fig.show(renderer='browser')

In [245]:
# Since there is no out-of-plane motion, we can plot the trajectories in the along-track radial plane
fig = go.Figure(data=[go.Scatter(x=[0],
                                 y=[100],
                                 mode = 'markers',
                                 name = 'Radial Axis',
                                 marker= dict(size=10, symbol='arrow-up', color="red")), 
                        go.Scatter(x=[100],
                                   y=[0],
                                   mode = 'markers',
                                   name = 'Along-Track Axis',
                                   marker= dict(size=10, symbol='arrow-right', color="green")),
                        go.Scatter(x=[0, 100],
                                   y=[0, 0],
                                   mode = 'lines',
                                   name = 'Along-Track Axis',
                                   line = dict(width=1, color="green"), 
                                   showlegend=False),
                        go.Scatter(x=[0, 0],
                                   y=[0, 100],
                                   mode = 'lines',
                                   name = 'Radial Axis',
                                   line = dict(width=1, color="red"), 
                                   showlegend=False),
                        go.Scatter(x=[0],
                                   y=[0],
                                   mode = 'markers',
                                   name = 'Hill Frame Origin (Leader)',
                                   marker = dict(size=10, symbol='circle-x-open', color="blue"))])

theta = np.linspace(0, 2*np.pi, 100)
x_keep_out = keep_out_rad * np.cos(theta)
y_keep_out = keep_out_rad * np.sin(theta)
fig.add_trace(go.Scatter(x=x_keep_out,y=y_keep_out, fill="toself", name="Keep-Out Sphere", mode="lines", line=dict(color="Red"), opacity=0.7))

fig.add_trace(go.Scatter(x=state_chaser_hill_hist[:rndz_idxs[1]+1,1],
                        y=state_chaser_hill_hist[:rndz_idxs[1]+1,0],
                        mode = 'markers',
                        name = 'Chaser Trajectory',
                        marker = dict(size=5, 
                                      color=times[:rndz_idxs[1]+1], 
                                      colorscale="Matter")))

fig.add_trace(go.Scatter(x=[state_chaser_hill_hist[0,1]],
                        y=[state_chaser_hill_hist[0,0]],
                        mode = 'markers',
                        name = 'Chaser Initial Position',
                        marker = dict(size=9)))

fig.add_trace(go.Scatter(x=[state_chaser_hill_hist[rndz_idxs[0]+1,1]],
                        y=[state_chaser_hill_hist[rndz_idxs[0]+1,0]],
                        mode = 'markers',
                        name = 'Chaser Position at Rendezvous Init',
                        marker = dict(size=7)))

fig.add_trace(go.Scatter(x=[state_chaser_hill_hist[rndz_idxs[1]+1,1]],
                        y=[state_chaser_hill_hist[rndz_idxs[1]+1,0]],
                        mode = 'markers',
                        name = 'Chaser Position at Rendezvous End',
                        marker = dict(size=7)))

fig.add_trace(go.Scatter(x=[waypoints[0,1]],
                        y=[waypoints[0,0]],
                        mode = 'markers',
                        name = 'Waypoint 0',
                        marker = dict(size=5)))

fig.update_layout(#title="Rendezvous and Docking Trajectory Rendezvous Phase (Radial/Along-Track Plane)",
                  xaxis_title='Along-Track (km)',
                  xaxis=dict(tickmode='linear', tick0=0, dtick=100),
                  yaxis=dict(tickmode='linear', tick0=0, dtick=100),
                  yaxis_title='Radial (km)',
                  yaxis_scaleanchor="x", yaxis_range=[-keep_out_rad*1.2, keep_out_rad*1.2],)

fig.show(renderer='browser')

In [246]:
# Since there is no out-of-plane motion, we can plot the trajectories in the along-track radial plane
fig = go.Figure(data=[go.Scatter(x=[0],
                                 y=[100],
                                 mode = 'markers',
                                 name = 'Radial Axis',
                                 marker= dict(size=10, symbol='arrow-up', color="red")), 
                        go.Scatter(x=[100],
                                   y=[0],
                                   mode = 'markers',
                                   name = 'Along-Track Axis',
                                   marker= dict(size=10, symbol='arrow-right', color="green")),
                        go.Scatter(x=[0, 100],
                                   y=[0, 0],
                                   mode = 'lines',
                                   name = 'Along-Track Axis',
                                   line = dict(width=1, color="green"), 
                                   showlegend=False),
                        go.Scatter(x=[0, 0],
                                   y=[0, 100],
                                   mode = 'lines',
                                   name = 'Radial Axis',
                                   line = dict(width=1, color="red"), 
                                   showlegend=False),
                        go.Scatter(x=[0],
                                   y=[0],
                                   mode = 'markers',
                                   name = 'Hill Frame Origin (Leader)',
                                   marker = dict(size=10, symbol='circle-x-open', color="blue"))])

appch_corridor_x = np.array([keep_out_rad, 0, keep_out_rad])
appch_corridor_y = np.array([-keep_out_rad*np.sin(np.radians(approach_cone_angle/2)), 0, keep_out_rad*np.sin(np.radians(approach_cone_angle/2))])

extended_corridor_x = np.array([keep_out_rad, 1.2*keep_out_rad])
extended_corridor_y = np.array([keep_out_rad*np.sin(np.radians(approach_cone_angle/2)), 1.2*keep_out_rad*np.sin(np.radians(approach_cone_angle/2))])

fig.add_trace(go.Scatter(x=appch_corridor_x + waypoints[1,1], 
                         y=appch_corridor_y + waypoints[1,0], 
                         fill="toself", name="Docking Corridor", mode="lines", marker=dict(color="green"), opacity=0.7))
fig.add_trace(go.Scatter(x=extended_corridor_x + waypoints[1,1], 
                         y=extended_corridor_y + waypoints[1,0], 
                         name="Extended Corridor", mode="lines", line=dict(color="green",dash="dash"), opacity=0.7,showlegend=True))
fig.add_trace(go.Scatter(x=extended_corridor_x + waypoints[1,1], 
                         y=-extended_corridor_y + waypoints[1,0], 
                         name="Extended Corridor", mode="lines", line=dict(color="green",dash="dash"), opacity=0.7,showlegend=False))

fig.add_trace(go.Scatter(x=state_chaser_hill_hist[rndz_idxs[0]:rndz_idxs[1]+1,1],
                        y=state_chaser_hill_hist[rndz_idxs[0]:rndz_idxs[1]+1,0],
                        mode = 'markers',
                        name = 'Chaser Trajectory',
                        marker = dict(size=5, 
                                      color=times[rndz_idxs[0]:rndz_idxs[1]+1], 
                                      colorscale="Matter")))

fig.add_trace(go.Scatter(x=[state_chaser_hill_hist[rndz_idxs[0]+1,1]],
                        y=[state_chaser_hill_hist[rndz_idxs[0]+1,0]],
                        mode = 'markers',
                        name = 'Chaser Position at Rendezvous Init',
                        marker = dict(size=7)))

fig.add_trace(go.Scatter(x=[state_chaser_hill_hist[rndz_idxs[1]+1,1]],
                        y=[state_chaser_hill_hist[rndz_idxs[1]+1,0]],
                        mode = 'markers',
                        name = 'Chaser at Rendezvous End',
                        marker = dict(size=7)))

fig.add_trace(go.Scatter(x=[waypoints[0,1]],
                        y=[waypoints[0,0]],
                        mode = 'markers',
                        name = 'Waypoint 0',
                        marker = dict(size=5)))

fig.add_trace(go.Scatter(x=[waypoints[1,1]],
                        y=[waypoints[1,0]],
                        mode = 'markers',
                        name = 'Waypoint 1',
                        marker = dict(size=5)))

fig.update_layout(title="Rendezvous and Docking Trajectory Rendezvous Phase (Radial/Along-Track Plane)",
                  xaxis_title='Along-Track (m)',
                  xaxis=dict(tickmode='linear', tick0=0, dtick=100),
                  yaxis=dict(tickmode='linear', tick0=0, dtick=100),
                  yaxis_title='Radial (m)',
                  yaxis_scaleanchor="x")

fig.show(renderer='browser')

In [247]:
# Since there is no out-of-plane motion, we can plot the trajectories in the along-track radial plane
fig = go.Figure(data=[go.Scatter(x=[0],
                                 y=[100],
                                 mode = 'markers',
                                 name = 'Radial Axis',
                                 marker= dict(size=10, symbol='arrow-up', color="red")), 
                        go.Scatter(x=[100],
                                   y=[0],
                                   mode = 'markers',
                                   name = 'Along-Track Axis',
                                   marker= dict(size=10, symbol='arrow-right', color="green")),
                        go.Scatter(x=[0, 100],
                                   y=[0, 0],
                                   mode = 'lines',
                                   name = 'Along-Track Axis',
                                   line = dict(width=1, color="green"), 
                                   showlegend=False),
                        go.Scatter(x=[0, 0],
                                   y=[0, 100],
                                   mode = 'lines',
                                   name = 'Radial Axis',
                                   line = dict(width=1, color="red"), 
                                   showlegend=False),
                        go.Scatter(x=[0],
                                   y=[0],
                                   mode = 'markers',
                                   name = 'Hill Frame Origin (Leader)',
                                   marker = dict(size=10, symbol='circle-x-open', color="blue"))])

fig.add_trace(go.Scatter(x=appch_corridor_x + waypoints[1,1], 
                         y=appch_corridor_y + waypoints[1,0], 
                         fill="toself", name="Docking Corridor", mode="lines", marker=dict(color="green"), opacity=0.7))
fig.add_trace(go.Scatter(x=extended_corridor_x + waypoints[1,1], 
                         y=extended_corridor_y + waypoints[1,0], 
                         name="Extended Corridor", mode="lines", line=dict(color="green",dash="dash"), opacity=0.7,showlegend=True))
fig.add_trace(go.Scatter(x=extended_corridor_x + waypoints[1,1], 
                         y=-extended_corridor_y + waypoints[1,0], 
                         name="Extended Corridor", mode="lines", line=dict(color="green",dash="dash"), opacity=0.7,showlegend=False))

fig.add_trace(go.Scatter(x=state_chaser_hill_hist[rndz_idxs[1]:rndz_idxs[2]+1,1],
                        y=state_chaser_hill_hist[rndz_idxs[1]:rndz_idxs[2]+1,0],
                        mode = 'markers',
                        name = 'Chaser Trajectory',
                        marker = dict(size=5, 
                                      color=times[rndz_idxs[1]:rndz_idxs[2]+1], 
                                      colorscale="Matter")))

fig.add_trace(go.Scatter(x=[state_chaser_hill_hist[rndz_idxs[1]+1,1]],
                        y=[state_chaser_hill_hist[rndz_idxs[1]+1,0]],
                        mode = 'markers',
                        name = 'Chaser Position at Docking Init',
                        marker = dict(size=7)))

fig.add_trace(go.Scatter(x=[state_chaser_hill_hist[rndz_idxs[2]+1,1]],
                        y=[state_chaser_hill_hist[rndz_idxs[2]+1,0]],
                        mode = 'markers',
                        name = 'Chaser Final Position',
                        marker = dict(size=7)))

fig.add_trace(go.Scatter(x=[waypoints[0,1]],
                        y=[waypoints[0,0]],
                        mode = 'markers',
                        name = 'Waypoint 0',
                        marker = dict(size=10)))

fig.add_trace(go.Scatter(x=[waypoints[1,1]],
                        y=[waypoints[1,0]],
                        mode = 'markers',
                        name = 'Waypoint 1',
                        marker = dict(size=10)))

fig.update_layout(title="Rendezvous and Docking Trajectory Docking Phase (Radial/Along-Track Plane)",
                  xaxis_title='Along-Track (m)',
                  xaxis=dict(tickmode='linear', tick0=0, dtick=100),
                  yaxis=dict(tickmode='linear', tick0=0, dtick=100),
                  yaxis_title='Radial (m)',
                  yaxis_scaleanchor="x")

fig.show(renderer='browser')

In [248]:
fig     = go.Figure(go.Scatter(x=times[rndz_idxs[0]:rndz_idxs[1]+1],
                            y=inputs[rndz_idxs[0]:rndz_idxs[1]+1,0],
                            mode = 'lines',
                            name = 'Acceleration Cmd (X)',
                            line = dict(width=3, color="red")))
fig.add_trace(go.Scatter(x=times[rndz_idxs[0]:rndz_idxs[1]+1],
                            y=inputs[rndz_idxs[0]:rndz_idxs[1]+1,1],
                            mode = 'lines',
                            name = 'Acceleration Cmd (Y)',
                            line = dict(width=3, color="green")))
fig.add_trace(go.Scatter(x=times[rndz_idxs[0]:rndz_idxs[1]+1],
                            y=inputs[rndz_idxs[0]:rndz_idxs[1]+1,2],
                            mode = 'lines',
                            name = 'Acceleration Cmd (Z)',
                            line = dict(width=3, color="blue")))
fig.update_layout(title="Acceleration Commands Rendezvous Phase",
                  xaxis_title='Time (s)',
                  yaxis_title='Acceleration (m/s^2)', yaxis_range=[-0.05, 0.05])
fig.add_vline(x=times[rndz_idxs[0]], line_width=2, line_dash="dash", line_color="black", annotation_text="Rendezvous Start", annotation_position="top right")
fig.add_vline(x=times[rndz_idxs[1]], line_width=2, line_dash="dash", line_color="black", annotation_text="Rendezvous End", annotation_position="top left")
fig.add_hline(y=0.032, line_width=2, line_dash="dash", line_color="red", annotation_text="Acceleration Cmd Limit", annotation_position="top left")
fig.add_hline(y=-0.032, line_width=2, line_dash="dash", line_color="red", annotation_text="Acceleration Cmd Limit", annotation_position="bottom left")
fig.show(renderer='browser')

fig     = go.Figure(go.Scatter(x=times[rndz_idxs[1]:rndz_idxs[2]+1],
                            y=inputs[rndz_idxs[1]:rndz_idxs[2]+1,0],
                            mode = 'lines',
                            name = 'Acceleration Cmd (X)',
                            line = dict(width=3, color="red")))
fig.add_trace(go.Scatter(x=times[rndz_idxs[1]:rndz_idxs[2]+1],
                            y=inputs[rndz_idxs[1]:rndz_idxs[2]+1,1],
                            mode = 'lines',
                            name = 'Acceleration Cmd (Y)',
                            line = dict(width=3, color="green")))
fig.add_trace(go.Scatter(x=times[rndz_idxs[1]:rndz_idxs[2]+1],
                            y=inputs[rndz_idxs[1]:rndz_idxs[2]+1,2],
                            mode = 'lines',
                            name = 'Acceleration Cmd (Z)',
                            line = dict(width=3, color="blue")))
fig.update_layout(title="Acceleration Commands Docking Phase",
                  xaxis_title='Time (s)',
                  yaxis_title='Acceleration (m/s^2)', yaxis_range=[-0.05, 0.05])
fig.add_vline(x=times[rndz_idxs[1]], line_width=2, line_dash="dash", line_color="black", annotation_text="Docking Start", annotation_position="top right")
fig.add_vline(x=times[rndz_idxs[2]], line_width=2, line_dash="dash", line_color="black", annotation_text="Docking End", annotation_position="top left")
fig.add_hline(y=0.032, line_width=2, line_dash="dash", line_color="red", annotation_text="Acceleration Cmd Limit", annotation_position="top left")
fig.add_hline(y=-0.032, line_width=2, line_dash="dash", line_color="red", annotation_text="Acceleration Cmd Limit", annotation_position="bottom left")

fig.show(renderer='browser')

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, x_title="Time (s)")
fig.add_trace(go.Scatter(x=times, y=state_chaser_hill_hist[:,0],
                         mode = 'lines', name = 'Relative Position (X)',
                         line = dict(width=3, color="red")),row=1, col=1)
fig.add_trace(go.Scatter(x=times, y=state_chaser_hill_hist[:,1],
                         mode = 'lines', name = 'Relative Position (Y)',
                         line = dict(width=3, color="green")), row=1, col=1)
fig.add_trace(go.Scatter(x=times, y=state_chaser_hill_hist[:,2],
                         mode = 'lines', name = 'Relative Position (Z)',
                         line = dict(width=3, color="blue")), row=1, col=1)
fig.add_trace(go.Scatter(x=times, y=state_chaser_hill_hist[:,3],
                         mode = 'lines', name = 'Relative Velocity (X)',
                         line = dict(width=3, color="cyan")),row=2, col=1)
fig.add_trace(go.Scatter(x=times, y=state_chaser_hill_hist[:,4],
                         mode = 'lines', name = 'Relative Velocity (Y)',
                         line = dict(width=3, color="magenta")), row=2, col=1)
fig.add_trace(go.Scatter(x=times, y=state_chaser_hill_hist[:,5],
                         mode = 'lines', name = 'Relative Velocity (Z)',
                         line = dict(width=3, color="yellow")), row=2, col=1)
fig.update_layout(#title="Relative States in Hill Frame",
                  xaxis_title='Time (s)', font=dict(size=20))
fig.update_yaxes(title_text="Relative Position (m)", row=1, col=1)
fig.update_yaxes(title_text="Relative Velocity (m/s)", row=2, col=1)
fig.add_vline(x=times[rndz_idxs[0]], line_width=2, line_dash="dash", line_color="black", annotation_text="Rendezvous Start", annotation_position="bottom right")
fig.add_vline(x=times[rndz_idxs[1]], line_width=2, line_dash="dash", line_color="black", annotation_text="Rendezvous End", annotation_position="bottom left")
fig.add_vline(x=times[rndz_idxs[2]], line_width=2, line_dash="dash", line_color="black", annotation_text="Docking End", annotation_position="bottom right")
fig.show(renderer='browser')

In [249]:
fig     = go.Figure(go.Scatter(x=times[rndz_idxs[0]:rndz_idxs[1]+1],
                            y=np.linalg.norm(state_chaser_hill_hist[rndz_idxs[0]:rndz_idxs[1]+1, 0:3], axis=1),
                            mode = 'lines',
                            name = 'Range to Target',
                            line = dict(width=3)))
fig.add_hline(y=keep_out_rad, line_width=2, line_dash="dash", line_color="red", annotation_text="Keep-Out Radius", annotation_position="bottom left")
fig.add_vline(x=times[rndz_idxs[0]], line_width=2, line_dash="dash", line_color="black", annotation_text="Rendezvous Start", annotation_position="top right")
fig.add_vline(x=times[rndz_idxs[1]], line_width=2, line_dash="dash", line_color="black", annotation_text="Rendezvous End", annotation_position="top left")
fig.update_layout(title="Range Rendezvous Phase",
                  xaxis_title='Time (s)',
                  yaxis_title='Distance (m)')
fig.show(renderer='browser')

In [250]:
v_axis = np.zeros_like(state_chaser_hill_hist[rndz_idxs[1]:rndz_idxs[2]+1, 0])
for i in range(len(v_axis)):
    v_axis[i] = np.abs(np.dot(state_chaser_hill_hist[rndz_idxs[1]+i, 3:6], approach_direction))

fig     = go.Figure(go.Scatter(x=times[rndz_idxs[1]:rndz_idxs[2]+1],
                            y= v_axis,
                            mode = 'lines',
                            name = 'Velocity along Docking Axis',
                            line = dict(width=3)))
fig.add_hline(y=1, line_width=2, line_dash="dash", line_color="red", annotation_text="Docking Axis Velocity Limit", annotation_position="bottom left")
fig.add_vline(x=times[rndz_idxs[1]], line_width=2, line_dash="dash", line_color="black", annotation_text="Docking Start", annotation_position="bottom right")
fig.add_vline(x=times[rndz_idxs[2]], line_width=2, line_dash="dash", line_color="black", annotation_text="Docking End", annotation_position="bottom left")
fig.update_layout(title="Closure Rate Along Docking Axis",
                  xaxis_title='Time (s)',
                  yaxis_title='Velocity (m/s)')
fig.show(renderer='browser')

In [251]:
def make_img_frame(i, rndz_idxs):
    frame = [go.Scatter(x=[0], y=[100], mode = 'markers',name = 'Radial Axis',
                        marker= dict(size=10, symbol='arrow-up', color="red")), 
            go.Scatter(x=[100], y=[0], mode = 'markers', name = 'Along-Track Axis',
                       marker= dict(size=10, symbol='arrow-right', color="green")),
            go.Scatter(x=[0, 100], y=[0, 0], mode = 'lines', name = 'Along-Track Axis',
                       line = dict(width=1, color="green"), showlegend=False),
            go.Scatter(x=[0, 0], y=[0, 100], mode = 'lines', name = 'Radial Axis',
                       line = dict(width=1, color="red"), showlegend=False),
            go.Scatter(x=[0],y=[0], mode = 'markers', name = 'Hill Frame Origin (Leader)',
                        marker = dict(size=10, symbol='circle-x-open', color="blue"))]
    
    theta = np.linspace(0, 2*np.pi, 100)
    x_keep_out = keep_out_rad * np.cos(theta)
    y_keep_out = keep_out_rad * np.sin(theta)
    if i > rndz_idxs[0] and i < rndz_idxs[1]:
        frame.append(go.Scatter(x=x_keep_out,y=y_keep_out, 
                                    fill="toself", name="Keep-Out Sphere", mode="lines", 
                                    line=dict(color="Red"), opacity=0.7))
    else:
        frame.append(go.Scatter(x=[0],y=[0], mode = 'markers', name = 'Hill Frame Origin (Leader)',
                        marker = dict(size=10, symbol='circle-x-open', color="blue"), showlegend=False))

    if i > rndz_idxs[1] and i < rndz_idxs[2]:
        frame.append(go.Scatter(x=appch_corridor_x + waypoints[1,1], 
                                y=appch_corridor_y + waypoints[1,0], 
                                fill="toself", name="Docking Corridor", mode="lines", marker=dict(color="green"), opacity=0.7, showlegend=False))
        frame.append(go.Scatter(x=extended_corridor_x + waypoints[1,1], 
                                y=extended_corridor_y + waypoints[1,0], 
                                name="Extended Corridor", mode="lines", line=dict(color="green",dash="dash"), opacity=0.7,showlegend=False))
        frame.append(go.Scatter(x=extended_corridor_x + waypoints[1,1], 
                                y=-extended_corridor_y + waypoints[1,0], 
                                name="Extended Corridor", mode="lines", line=dict(color="green",dash="dash"), opacity=0.7,showlegend=False))
    else:
        frame.append(go.Scatter(x=[0],y=[0], mode = 'markers', name = 'Hill Frame Origin (Leader)',
                        marker = dict(size=10, symbol='circle-x-open', color="blue"), showlegend=False))
        frame.append(go.Scatter(x=[0],y=[0], mode = 'markers', name = 'Hill Frame Origin (Leader)',
                        marker = dict(size=10, symbol='circle-x-open', color="blue"), showlegend=False))
        frame.append(go.Scatter(x=[0],y=[0], mode = 'markers', name = 'Hill Frame Origin (Leader)',
                        marker = dict(size=10, symbol='circle-x-open', color="blue"), showlegend=False))
    
    frame.append(go.Scatter(x=state_chaser_hill_hist[0:i,1], y=state_chaser_hill_hist[0:i,0],
                        mode = 'markers', name = 'Chaser Trajectory',
                        marker = dict(size=5, color=times[0:i], colorscale="Matter")))
    
    if np.linalg.norm(state_chaser_hill_hist[i,3:4]) == 0:
        arrow_angle = 270
    else:
        arrow_angle = np.degrees(np.arctan2(state_chaser_hill_hist[i,4], state_chaser_hill_hist[i,3]))
    
    frame.append(go.Scatter(x=[state_chaser_hill_hist[i,1]],
                        y=[state_chaser_hill_hist[i,0]],
                        mode = 'markers', name = 'Chaser Position',
                        marker=dict(symbol="arrow-wide", size=20, angle=arrow_angle)))
    
    frame.append(go.Scatter(x=[waypoints[0,1]],
                        y=[waypoints[0,0]],
                        mode = 'markers',
                        name = 'Waypoint 0',
                        marker = dict(size=7)))
    
    frame.append(go.Scatter(x=[waypoints[1,1]],
                        y=[waypoints[1,0]],
                        mode = 'markers',
                        name = 'Waypoint 1',
                        marker = dict(size=7)))
    
    return frame

In [252]:
frames      = 200
frame_rate  = int(len(times)/frames)
frame_data  = make_img_frame(0, rndz_idxs)
animation   = go.Figure(data=frame_data)
animation.update(frames=[go.Frame(data=[*make_img_frame(i*frame_rate, rndz_idxs)], 
                                traces=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]) 
                                for i in tqdm.tqdm(range(int(len(times)/frame_rate)))])
animation.update_layout(title=dict(text="Rendezvous and Docking Trajectory", font=dict(size=20)),
                      transition = {'duration': 1},
                      updatemenus=[dict(type="buttons",buttons=[dict(label="Play",method="animate",
                                                args=[None, {"frame": {"duration": 1, "redraw": True}}])])], 
                                                xaxis=dict(range=[-2.7*keep_out_rad, 2.7*keep_out_rad], tickmode='linear', tick0=0, dtick=100, title="Along-Track (m)"),
                                                yaxis=dict(range=[-1.2*keep_out_rad,1.2*keep_out_rad], tickmode='linear', tick0=0, dtick=100, title="Radial (m)"), 
                                                yaxis_scaleanchor="x")
animation.show(renderer='browser')

100%|██████████| 222/222 [00:01<00:00, 113.04it/s]


In [253]:
def make_3d_frame(idx):
    # Plot the entire simulation trajectory in the Hill frame (Leader)
    frame     = [go.Scatter3d(x=state_chaser_hill_hist[0:idx,0]/1000, 
                                        y=state_chaser_hill_hist[0:idx,1]/1000, 
                                        z=state_chaser_hill_hist[0:idx,2]/1000, 
                                        mode = 'lines', 
                                        name = 'Chaser Trajectory', 
                                        line = dict(width=5, 
                                                    color=times, 
                                                    colorscale= 'Matter'), showlegend=False)]

    Dragon_points_t = np.hstack([Dragon_points, np.ones((Dragon_points.shape[0], 1))])  # Add a column of ones for homogeneous coordinates
    H = np.eye(4)
    body_x = np.array([0, 1, 0])
    des_orient = unit(waypoints[1, 0:3] - state_chaser_hill_hist[idx, 0:3])
    v = np.cross(body_x, des_orient)
    c = np.dot(body_x, des_orient)
    s = np.linalg.norm(v)

    if np.linalg.norm(state_chaser_hill_hist[idx, 0:3]) < keep_out_rad:
        R = np.array([[1, 0,  0],
                    [ 0, -1,  0],
                    [ 0, 0, 1]])
    else:
        K = skew(v)
        R = np.eye(3) + K + K @ K * ((1 - c) / (s**2))
    H[:3, 3] = state_chaser_hill_hist[idx,0:3]/1000
    H[:3, :3] = R
    Dragon_points_t = (H @ Dragon_points_t.T).T
    Dragon_points_t = Dragon_points_t[:, :3]  # Remove the homogeneous coordinate
    
    frame.append(go.Mesh3d(x=Dragon_points_t[:,0], 
                           y=Dragon_points_t[:,1]+2.4/1000, 
                           z=Dragon_points_t[:,2],
                           i=vertices_dragon[:,0], 
                           j=vertices_dragon[:,1], 
                           k=vertices_dragon[:,2],
                           color="snow", 
                           lighting=dict(diffuse=0.5), showlegend=False))

    frame.append(go.Mesh3d(x=ISS_points[:,0],
                            y=ISS_points[:,1],
                            z=ISS_points[:,2],
                            i=vertices[:,0],
                            j=vertices[:,1],
                            k=vertices[:,2],
                            alphahull=10,
                            opacity=1.00,
                            color="silver", 
                            lighting=dict(diffuse=0.5), showlegend=False))
    

    sphere = make_sphere(0, 0, 0, keep_out_rad/1000)
    x, y, z = sphere[:,0], sphere[:,1], sphere[:,2]

    frame.append(go.Mesh3d(x=x, y=y, z=z, color="red", opacity=0.10, alphahull=0, name="Keep Out Sphere"))

    theta = np.linspace(0, 2*np.pi, 100)
    r = np.sin(np.radians(approach_cone_angle/2)) * keep_out_rad/1000
    x = r * np.cos(theta) + waypoints[1,0]/1000
    y = np.ones_like(x) * keep_out_rad/1000
    z = r * np.sin(theta) + waypoints[1,2]/1000

    x=np.append(x,waypoints[1,0]/1000)
    y=np.append(y,waypoints[1,1]/1000)
    z=np.append(z,waypoints[1,2]/1000)

    frame.append(go.Mesh3d(x=x, y=y, z=z, color="green", opacity=0.20, alphahull=0, name="Approach Corridor"))

    fig = go.Figure(data=frame)
    add_frame(fig, np.eye(3)/4)

    return fig.data

In [254]:
start       = 600
frames      = 100
frame_rate  = int(len(times[start:])/frames)
frame_data  = make_3d_frame(start)
animation   = go.Figure(data=frame_data)
animation.update(frames=[go.Frame(data=[*make_3d_frame(start+i*frame_rate)], 
                                traces=[0, 1, 2]) 
                                for i in tqdm.tqdm(range(int(len(times[start:])/frame_rate)))])
animation.update_layout(title=dict(text="Rendezvous and Docking Trajectory", font=dict(size=20)),
                      transition = {'duration': 1},
                      scene=dict(xaxis = dict(title='Radial (km)'), 
                                yaxis = dict(title='Along-Track (km)'),
                                zaxis = dict(title='Out-of-Plane (km)'), 
                                camera=dict(up=dict(x=1, y=0, z=0),eye=dict(x=0, y=0.75, z=-2.3))),
                      updatemenus=[dict(type="buttons",buttons=[dict(label="Play",method="animate",
                                                args=[None, {"frame": {"duration": 1, "redraw": True}}],
                                                )])],)
animation.show(renderer='browser')

100%|██████████| 145/145 [00:05<00:00, 27.52it/s]
